In [ ]:
import numpy as np
import pandas as pd
## load data/TX/tx_long.csv
data = pd.read_csv('data/TX/tx_long.csv')
print(data.head())

  studentID  meas  Iv1_state  Av1_state  Uv1_state  Co1_state  Co2_state  \
0  student1     8        3.0        3.0        4.0        4.0        4.0   
1  student1    10        3.0        3.0        2.0        2.0        3.0   
2  student1    12        3.0        2.0        3.0        2.0        2.0   
3  student1    15        4.0        3.0        3.0        2.0        2.0   
4  student1    17        3.0        2.0        3.0        3.0        4.0   

   Angst_abbruch_state  Angst_scheitern_state  Leist_verstehen_state  ...  \
0                  2.0                    4.0                    3.0  ...   
1                  1.0                    2.0                    4.0  ...   
2                  1.0                    2.0                    4.0  ...   
3                  4.0                    4.0                    3.0  ...   
4                  1.0                    2.0                    3.0  ...   

   PANN09_state  Block2_lerngruppe_state  Block2_abschreiben_state  \
0         

In [6]:
# Drop columns with >50% missing values
# Calculate missing percentage per column
missing_pct = data.isnull().mean()

# Columns to drop
threshold = 0.5
cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()
print(f"Columns to drop (>{int(threshold*100)}% missing): {len(cols_to_drop)}")
print(cols_to_drop)

# Show shape before
print('Shape before:', data.shape)

# Drop the columns
data_clean = data.drop(columns=cols_to_drop)

data_clean.to_csv('data/TX/tx_long_cleaned.csv', index=False)


Columns to drop (>50% missing): 8
['Block2_lerngruppe_state', 'Block2_abschreiben_state', 'Block2_durchhaltevermoegen_state', 'Block2_vorNachbereitung_state', 'Block2_zeitaufwand_state', 'Block2_anwesenheit_state', 'Block2_gruppe_state', 'Summe']
Shape before: (3187, 30)


## Preprocessing plan for CatBoost dropout model

- Build target: predict next period dropout using current-row features. Create `next_event` = groupby(studentID)['event'].shift(-1) and drop rows where it's missing.
- Strictly avoid leakage: all lag/rolling features use past data only (shifted by 1).
- Create group-wise lags (lag 1,2,3) and rolling means for numeric features.
- Add time-based features: days since previous measurement, days since first measurement, cumulative counts.
- Handle missingness: forward-fill within student groups, then median (numerics) / mode or 'missing' (categoricals).
- Leave categorical columns as string/category so CatBoost can accept them; pass `cat_features` when training.
- Save preprocessed dataset to `data/TX/tx_long_preprocessed.csv` for model training.

Run the preprocessing cell next to generate features and verify no leakage before training.

In [12]:
data = pd.read_csv('data/TX/tx_long_cleaned.csv')

# Coerce to numeric; invalid entries become NaN and we'll drop them (or you can choose another strategy)
data['meas'] = pd.to_numeric(data['meas'], errors='coerce')
na_meas = int(data['meas'].isna().sum())
if na_meas > 0:
    print(f'Warning: {na_meas} rows have invalid/non-numeric meas and will be dropped')
    data = data[~data['meas'].isna()].copy()

# Convert to integer week numbers (if they are floats from CSV)
data['meas'] = data['meas'].astype(int)

# Sort by student and meas (week index)
data = data.sort_values(['studentID', 'meas']).reset_index(drop=True)

# 1) find first event time per student (earliest meas with event==1)
first_evt = (
    data[data['event'] == 1]
        .groupby('studentID')['meas']
        .min()
        .rename('drpopout')
)
# merge first-event info back to data (NaN means never dropped)
data = data.merge(first_evt, on='studentID', how='left')

# 2) keep only at-risk rows: rows strictly before the first event (or all rows if student never dropped)
atrisk = data[(data['drpopout'].isna()) | (data['meas'] < data['drpopout'])].copy()
print(f"Rows before filtering at-risk: {len(data)}, after at-risk filter: {len(atrisk)}")

# 3) build target: next-period dropout (t -> t+1)
# compute on the full data so shift aligns with real next measurement; then filter keeps only at-risk rows
data['next_event'] = data.groupby('studentID')['event'].shift(-1)

# Now restrict to atrisk rows and drop rows where next_event is missing (last observation for a student)
data = atrisk.merge(data[['studentID','meas','next_event']], on=['studentID','meas'], how='left')

# Drop rows where we cannot build the target (last measurement per student)
data = data[~data['next_event'].isna()].copy()
data['next_event'] = data['next_event'].astype(int)

# Identify numeric and categorical feature candidates (exclude identifiers and target)
exclude_cols = {'studentID', 'meas', 'event', 'next_event', 'drpopout'}
num_cols = [c for c in data.select_dtypes(include=[np.number]).columns if c not in exclude_cols]
cat_cols = [c for c in data.select_dtypes(include=['object','category']).columns if c not in exclude_cols]

print(f'Numeric features detected: {num_cols}')
print(f'Categorical features detected: {cat_cols}')

# Create lags and rolling features for numeric columns (use only past values)
lags = [1,2,3]
for lag in lags:
    for c in num_cols:
        data[f'{c}_lag{lag}'] = data.groupby('studentID')[c].shift(lag)

# Rolling means using past values (shift by 1 to exclude current row)
windows = [2,3,5]
for w in windows:
    for c in num_cols:
        # shift(1) ensures only past values included
        data[f'{c}_roll_mean_{w}'] = (
            data.groupby('studentID')[c].shift(1)
                .rolling(window=w, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)
        )

# Time-delta features using week-index (meas is integer week number)
# previous meas (week index)
data['meas_prev'] = data.groupby('studentID')['meas'].shift(1)
# weeks since previous measurement (integer weeks). first observation will be NaN -> fill with 0
data['weeks_since_prev'] = (data['meas'] - data['meas_prev']).fillna(0).astype(int)
# weeks since first measurement
data['weeks_since_first'] = (data['meas'] - data.groupby('studentID')['meas'].transform('first')).fillna(0).astype(int)
# observation count
data['obs_count'] = data.groupby('studentID').cumcount() + 1

# Forward-fill within each student to propagate most recent past info
# Note: forward-fill will not fill backward in time; since features are constructed from past, this is OK
data = data.groupby('studentID').apply(lambda df: df.ffill()).reset_index(drop=True)

# Impute remaining numerical NAs with median
for c in data.select_dtypes(include=[np.number]).columns:
    if c in exclude_cols:
        continue
    if data[c].isna().any():
        med = data[c].median()
        data[c] = data[c].fillna(med)

# For categorical columns, fill NA with a placeholder and convert to category
for c in cat_cols:
    data[c] = data[c].fillna('missing').astype('category')

# Final feature list (exclude ids and raw target)
feature_cols = [c for c in data.columns if c not in ['studentID', 'meas', 'event', 'next_event', 'meas_prev', 'drpopout']]
print('Number of features:', len(feature_cols))

# Save preprocessed data
out_path = 'data/TX/tx_long_preprocessed.csv'
data.to_csv(out_path, index=False)
print(f'Preprocessed data saved to: {out_path}')
print('Resulting data shape:', data.shape)

# Show a quick preview of top features and class balance
print('\nSample columns:', feature_cols[:20])
print('\nClass distribution for next_event:')
print(data['next_event'].value_counts(normalize=False))

# Reminder: when training CatBoost, pass categorical feature names in `cat_features` (cat_cols)
print('\nCategorical features to pass to CatBoost:', cat_cols)

Rows before filtering at-risk: 3187, after at-risk filter: 3065
Numeric features detected: ['Iv1_state', 'Av1_state', 'Uv1_state', 'Co1_state', 'Co2_state', 'Angst_abbruch_state', 'Angst_scheitern_state', 'Leist_verstehen_state', 'Leist_bearbeiten_state', 'Leist_stress_state', 'Leist_ueberfordert_state', 'Wiss_kommilitonen_state', 'Wiss_mathe_state', 'PANP01_state', 'PANP05_state', 'PANP08_state', 'PANN01_state', 'PANN05_state', 'PANN09_state']
Categorical features detected: []


C:\Users\thinkpad\AppData\Local\Temp\ipykernel_3948\2153889892.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{c}_roll_mean_{w}'] = (
C:\Users\thinkpad\AppData\Local\Temp\ipykernel_3948\2153889892.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{c}_roll_mean_{w}'] = (
C:\Users\thinkpad\AppData\Local\Temp\ipykernel_3948\2153889892.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co

Number of features: 136
Preprocessed data saved to: data/TX/tx_long_preprocessed.csv
Resulting data shape: (2991, 142)

Sample columns: ['Iv1_state', 'Av1_state', 'Uv1_state', 'Co1_state', 'Co2_state', 'Angst_abbruch_state', 'Angst_scheitern_state', 'Leist_verstehen_state', 'Leist_bearbeiten_state', 'Leist_stress_state', 'Leist_ueberfordert_state', 'Wiss_kommilitonen_state', 'Wiss_mathe_state', 'PANP01_state', 'PANP05_state', 'PANP08_state', 'PANN01_state', 'PANN05_state', 'PANN09_state', 'Iv1_state_lag1']

Class distribution for next_event:
next_event
0    2948
1      43
Name: count, dtype: int64

Categorical features to pass to CatBoost: []


In [ ]:
## load data/T0/aist.csv
bg = pd.read_csv('data/T0/background.csv')
bg['age'] = 2025 - bg['gebu']   
bg['age_centered'] = bg['age'] - bg['age'].mean()
## delete gebu column
bg = bg.drop(columns=['gebu'])
# save processed background data
bg.to_csv('data/T0/background_processed.csv', index=False)

In [26]:
big_five = pd.read_csv('data/T0/bfi.csv')
print(big_five.head())

    studentID  bfi_ex  bfi_ve  bfi_ge  bfi_ne  bfi_of
0    student1    2.67    2.33    4.33    4.33    4.00
1   student10    2.33    2.33    3.33    2.67    3.67
2  student100    2.00    3.33    2.33    1.33    4.00
3  student101    1.67    2.67    2.67    3.33    4.67
4  student102    3.67    3.33    3.67    2.33    4.00


In [16]:
## merge all processed dataframes on studentID
bg = pd.read_csv('data/T0/background_processed.csv')
big_five = pd.read_csv('data/T0/bfi.csv')
iq_data = pd.read_csv('data/T0/iq_processed.csv')
math_tests = pd.read_csv('data/T0/math_test_processed.csv')
#concat all dataframes on studentID
merged = bg.merge(big_five, on='studentID', how='inner')
merged = merged.merge(iq_data, on='studentID', how='inner')
merged = merged.merge(math_tests, on='studentID', how='inner')
## save merged data
merged.to_csv('data/T0/merged_T0.csv', index=False)


In [ ]:
## merge T0 data into TX long data
tx_long = pd.read_csv('data/TX/tx_long_preprocessed.csv')
t0_t1 = tx_long.merge(merged, on='studentID', how='left')
t0_t1.to_csv('data/TX/tx_long_preprocessed_with_T0.csv', index=False)

## Training datasets for CatBoost dropout model

### Training dataset for T1 only model

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_recall_curve, average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.utils.class_weight import compute_class_weight

data = pd.read_csv('data/TX/tx_long_preprocessed.csv')
X = data[feature_cols]
y = data['next_event']
weights = compute_class_weight('balanced', classes=np.array([0,1]), y=y)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []
f1_scores = []
accuracy_scores = []
pr_auc_scores = []
for train_idx, val_idx in kf.split(X, y):
    train_pool = Pool(X.iloc[train_idx], y.iloc[train_idx])
    val_pool   = Pool(X.iloc[val_idx], y.iloc[val_idx])

    model = CatBoostClassifier(
        depth=6, learning_rate=0.05, iterations=500,
        l2_leaf_reg=5, class_weights=weights, random_strength=1.0, verbose=0
    )
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)
    y_prob = model.predict_proba(val_pool)[:,1]
    y_valid = y.iloc[val_idx]
    prec, rec, thr = precision_recall_curve(y_valid, y_prob)
    f1 = 2 * prec * rec / (prec + rec + 1e-9)
    best_t = thr[np.argmax(f1)]

    y_pred = (y_prob >= best_t).astype(int)
    auc = roc_auc_score(y_valid, y_prob)
    f1_val = f1_score(y_valid, y_pred)
    accuracy = accuracy_score(y_valid, y_pred)
    pr_auc = average_precision_score(y_valid, y_prob)
    auc_scores.append(auc)
    f1_scores.append(f1_val)
    accuracy_scores.append(accuracy)
    pr_auc_scores.append(pr_auc)
    

print(f"Mean AUC: {np.mean(auc_scores):.3f} ± {np.std(auc_scores):.3f}")
print(f"F1 Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
print(f"Accuracy: {np.mean(accuracy_scores):.3f} ± {np.std(accuracy_scores):.3f}")
print(f"PR-AUC: {np.mean(pr_auc_scores):.3f} ± {np.std(pr_auc_scores):.3f}")




Mean AUC: 0.780 ± 0.054
F1 Score: 0.185 ± 0.053
Accuracy: 0.971 ± 0.013
PR-AUC: 0.102 ± 0.055


### Training dataset for T1 + T0 model

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_recall_curve, average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

data = pd.read_csv('data/TX/tx_long_preprocessed_with_t0.csv')
feature_cols = [c for c in data.columns if c not in ['studentID', 'meas', 'event', 'next_event', 'meas_prev', 'drpopout']]
X = data[feature_cols]
y = data['next_event']
weights = compute_class_weight('balanced', classes=np.array([0,1]), y=y)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []
f1_scores = []
accuracy_scores = []
pr_auc_scores = []
for train_idx, val_idx in kf.split(X, y):
    train_pool = Pool(X.iloc[train_idx], y.iloc[train_idx])
    val_pool   = Pool(X.iloc[val_idx], y.iloc[val_idx])

    model = CatBoostClassifier(
        depth=5, learning_rate=0.05, iterations=500,
        l2_leaf_reg=8, class_weights=weights, random_strength=1.0, verbose=0
    )
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)
    y_prob = model.predict_proba(val_pool)[:,1]
    y_valid = y.iloc[val_idx]
    prec, rec, thr = precision_recall_curve(y_valid, y_prob)
    f1 = 2 * prec * rec / (prec + rec + 1e-9)
    best_t = thr[np.argmax(f1)]

    y_pred = (y_prob >= best_t).astype(int)
    auc = roc_auc_score(y_valid, y_prob)
    f1_val = f1_score(y_valid, y_pred)
    accuracy = accuracy_score(y_valid, y_pred)
    pr_auc = average_precision_score(y_valid, y_prob)
    auc_scores.append(auc)
    f1_scores.append(f1_val)
    accuracy_scores.append(accuracy)
    pr_auc_scores.append(pr_auc)
    

print(f"Mean AUC: {np.mean(auc_scores):.3f} ± {np.std(auc_scores):.3f}")
print(f"F1 Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
print(f"Accuracy: {np.mean(accuracy_scores):.3f} ± {np.std(accuracy_scores):.3f}")
print(f"PR-AUC: {np.mean(pr_auc_scores):.3f} ± {np.std(pr_auc_scores):.3f}")




Mean AUC: 0.759 ± 0.065
F1 Score: 0.300 ± 0.046
Accuracy: 0.973 ± 0.020
PR-AUC: 0.186 ± 0.039


In [ ]:
model.get_feature_importance(prettified=True).head(20)

,Feature Id,Importances
0,weeks_since_first,21.941374
1,math_test_centered,11.020500
2,weeks_since_prev,9.863662
3,age,5.841781
4,Wiss_kommilitonen_state,5.745033
5,Wiss_kommilitonen_state_roll_mean_5,5.647301
6,Angst_scheitern_state_lag1,5.294276
7,Leist_ueberfordert_state_lag1,4.514367
8,PANP08_state_roll_mean_3,4.211695
9,Co1_state,2.744937


## Scales and Ranges for survey features

In [23]:
import pandas as pd
## upload data/TX/data_scales.xlsx
scales = pd.read_excel('data/TX/data_scales.xlsx', sheet_name=None)
## columns of each sheet
for sheet_name, df in scales.items():
    print(f'Sheet: {sheet_name}, Columns: {df.columns.tolist()}')


Sheet: aist.csv, Columns: ['Table name in frontend', 'Allgemeiner Interessen-Struktur-Test', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
Sheet: background.csv, Columns: ['Table name in frontend', 'Profil / Hintergrundinformationen', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Sheet: bfi.csv, Columns: ['Table name in frontend', 'Persönlichkeitsskala (Big-Five)', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Sheet: iq.csv, Columns: ['Table name in frontend', 'Kognitive Fähigkeiten', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Sheet: kont.csv, Columns: ['Table name in frontend', 'Internale-Externale-Kontrollüberzeugung', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Sheet: math_test.csv, Columns: ['Table name in frontend', 'Fachwissenstest Mathematik', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Sheet: motivation.csv, Columns: ['Table name in frontend', 'Motivation', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Sheet: panas.csv, Columns: ['Table name in frontend', 'PANAS (Positive and Negative

### TX scales

In [24]:
tx_scale = scales['tx_long.csv']
tx_scale = tx_scale.iloc[4:].reset_index(drop=True)
## delete second column
tx_scale = tx_scale.drop(columns=[tx_scale.columns[1]])
# print(tx_scale.head())
## rename column names as Columns, scale, range, general_name, prompt_in_questionnaire
tx_scale.columns = ['Columns', 'scale', 'range', 'general_name', 'prompt_in_questionnaire']
# delete the rows where Columns is NaN
tx_scale = tx_scale[~tx_scale['Columns'].isna()]
## save tx_scale to csv
tx_scale.to_csv('data/TX/tx_scale.csv', index=False)

In [ ]:
TX_scales = pd.read_csv('data/TX/tx_scale.csv')
TX_features = [c for c in TX_scales['Columns'] if c not in ['studentID', 'meas', 'event', 'Summe']]
prompt_TX = {
    row['Columns']: row['prompt_in_questionnaire'] 
    for _, row in TX_scales.iterrows() 
    if row['Columns'] in TX_features
}
print()

### T0 Scales

In [ ]:

# delete first 5 rows and second column
bf1_scales = bf1_scales.iloc[5:].reset_index(drop=True)
bf1_scales = bf1_scales.drop(columns=[bf1_scales.columns[1]])
bf1_scales.columns = ['Columns', 'scale', 'range', 'general_name']
bf1_scales

,Columns,scale,range,general_name
0,bfi_ex,float,1 to 5,Extraversion
1,bfi_ve,float,1 to 5,Verträglichkeit
2,bfi_ge,float,1 to 5,Gewissenhaftigkeit
3,bfi_ne,float,1 to 5,Negative Emotionalität
4,bfi_of,float,1 to 5,Offenheit


In [ ]:
bf1_scales = scales['bfi.csv']
bg_scales = scales['background.csv']
iq_scales = scales['iq.csv']
math_test = scales['math_test.csv']
for sc_df, name in zip(
    [bf1_scales, bg_scales, iq_scales, math_test],
    ['bfi', 'background', 'iq', 'math_test']
):
    sc_df = sc_df.iloc[5:].reset_index(drop=True)
    sc_df = sc_df.drop(columns=[sc_df.columns[1]])
    sc_df.columns = ['Columns', 'scale', 'range', 'general_name']


,Table name in frontend,Persönlichkeitsskala (Big-Five),Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,Columns,NaN,scale,range,general name
4,studentID,NaN,string,student1:N,ID
5,bfi_ex,NaN,float,1 to 5,Extraversion
6,bfi_ve,NaN,float,1 to 5,Verträglichkeit
7,bfi_ge,NaN,float,1 to 5,Gewissenhaftigkeit
8,bfi_ne,NaN,float,1 to 5,Negative Emotionalität
9,bfi_of,NaN,float,1 to 5,Offenheit


In [ ]:
bf1 = pd.read_csv('data/T0/bfi_scale.csv')
bg = pd.read_csv('data/T0/background_scale.csv')
iq = pd.read_csv('data/T0/iq_scale.csv')
math = pd.read_csv('data/T0/math_test_scale.csv')
## merge all scales into a single dataframe
T0_scales = pd.concat([bg, bf1, iq, math], ignore_index=True)
# change the last variable in T0_scales['Columns'] =='fw_pkt' to 'math_total_score'
T0_scales.loc[T0_scales['Columns'] == 'fw_pkt', 'Columns'] = 'math_total_score'
## save all scales to csv
T0_scales.to_csv('data/T0/T0_scales.csv', index=False)